# Python Code for Convection (PyCCon) v.2.3
## @author: Lena Noack


v2.3 Recent Updates (May 2025):
include restart option; allow for variable aspect ratios; include simple subduction zone set-up; include partial melting and outgassing


# Short documentation

This code can be used for simple convection simulations in a 2D box or 2D sphere,
using both thermal and chemical convection (the latter can be solved either
with a field approach or with particles for better resolution), and can use
temperature- or pressure-dependent viscosity (using the FK or Arrhenius approximation).
Convection is incompresisble with the (Extended) Boussinesq approximation or compressible with TALA approximation.

- main.ipynb: This is the main routine, please adapt the simulation parameters in the parameters cell below, assigning here the non-default parameter settings (default values will be overwritten)
- solver.py: Solves the energy, mass, momentum and chemical conservation equations
- supp_functions: Different initialization and support function, particle treatment, output measurements, and visualization (adapt as needed)
- outgassing.py: Includes outgassing calculations to build an atmosphere
- input_default.py: Loads the default settings and parameters, do not change

## Import modules

In [3]:
import matplotlib.pyplot as plt
#import modules, execution needed only once
import numpy as np
from supp_functions import *
import examples # for different example parameter settings
import convection


## Initialize and start convection simulation

In [4]:
data = supp_functions()               # Load default simulations values

#########################################################################################
# Change simulation parameters here to overwrite default values set in input_default.py #
#########################################################################################

SimCase = "" # simulate parameter cases are defined in examples.py; if none is selected, use parameters below
             # Example simulations currently available: Melting | Plasticity | SubductionZone | Compressibility | Isoviscous
             #                                          Arrhenius | FK |ThermoChemical | CompositionInput | TestRK | TestLateralMovement

if SimCase=="":
    data.geom = 0 # 0 - box, 1 - cylinder, 2 - spherical annulus
    data.nl = 40
    data.nr = 40
    data.Ra = 1e4

    data.plot = 'Tvuw'

    data.output = 'Test/Test_box_isovisc_Ra'+str(int(data.Ra))+'_'+str(data.nl)+'x'+str(data.nr)

    data.Cdt = -1 # >0 Courant criterium (~1); <0 Delta criterium (~-1)
    data.dt_max = 1e-2
    data.figiter = 5
    data.figtime = 0 # every ... outputs create a new figure
    data.nbiter = 0 # if set to zero, then use tmax
    data.tmax = 0.3

    data.snap_iter = 25
    data.read_snap = False # Set to True to restart simulation from last-written snapshot

examples.get_sim_param(data,SimCase) # only used if SimCase is set to name of pre-defined example

if (data.e_gamma_T>1.0):
  data.gamma_T = np.log(data.e_gamma_T)
  data.gamma_p = np.log(data.e_gamma_p)

#################################
# Run the convection simulation #
#################################
convection.run(data)


TS:    1, OI:  3, t: 1.000E-06, dt: 1.000E-06, |v|: 7.5455, |T|: 0.5000, |Nu_t/b|: 1.9491/ 1.9492, VC:    1 --   0.469s
TS:    2, OI:  7, t: 9.976E-04, dt: 9.966E-04, |v|: 9.1208, |T|: 0.5000, |Nu_t/b|: 1.1223/ 1.1519, VC:    1 --   1.688s
TS:    3, OI: 21, t: 6.772E-03, dt: 5.774E-03, |v|:34.5689, |T|: 0.5004, |Nu_t/b|: 1.1637/ 1.2650, VC:    1 --   4.164s
TS:    4, OI: 11, t: 9.989E-03, dt: 3.217E-03, |v|:56.8864, |T|: 0.5007, |Nu_t/b|: 2.0448/ 2.1557, VC:    1 --   2.638s
TS:    5, OI:  7, t: 1.198E-02, dt: 1.986E-03, |v|:68.9396, |T|: 0.5008, |Nu_t/b|: 2.9283/ 2.9978, VC:    1 --   1.301s
TS:    6, OI:  6, t: 1.364E-02, dt: 1.669E-03, |v|:74.8665, |T|: 0.5009, |Nu_t/b|: 3.9104/ 3.9062, VC:    1 --   1.615s
TS:    7, OI:  7, t: 1.564E-02, dt: 2.000E-03, |v|:75.1621, |T|: 0.5008, |Nu_t/b|: 5.1822/ 5.0763, VC:    1 --   1.189s
TS:    8, OI: 11, t: 1.928E-02, dt: 3.640E-03, |v|:64.0975, |T|: 0.5003, |Nu_t/b|: 6.7320/ 6.5489, VC:    1 --   2.322s
TS:    9, OI: 10, t: 2.172E-02, dt: 2.43

## Create an animated gif of snapshot plots

dataoutput is either the output from the cells above, or in case that the simulation was run in the past, the name of the folder in which simulation files are stored can be assigned to dataoutput (e.g. "Test/Ra10000_128x32")

In [5]:
# Create animated gif:
# Adapted from https://pythonprogramming.altervista.org/png-to-gif/

from PIL import Image
import glob

dataoutput = data.output

# Create the frames
frames = []
imgs = glob.glob(dataoutput+"/*.png")
count = 0
limit = 1 #10
for i in imgs:
  if count%limit==0:
    new_frame = Image.open(i)
    frames.append(new_frame)
  count += 1

# Save into a GIF file that loops forever
frames[0].save(dataoutput+'/animation.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=300, loop=0)

## Plot atmosphere evolution (if melting and outgassing is turned on)

In [6]:
if data.Tsol0>0:
    #############################
    # Plot atmosphere over time #
    #############################

    fig,ax = plt.subplots(2,1,figsize=(10,5))

    ax[0].plot(data.time,np.array(data.M_atm),'k',lw=2,label="All")
    ax[0].plot(data.time,np.array(data.M_H2O),'b',label="H2O")
    ax[0].plot(data.time,np.array(data.M_H2),'--g',label="H2")
    ax[0].plot(data.time,np.array(data.M_CO2),'r',label="CO2")
    ax[0].plot(data.time,np.array(data.M_CO),'--y',label="CO")
    ax[0].legend()
    ax[0].set_xlabel("Time")
    ax[0].set_ylabel("Atmosphere in kg")

    ptot = np.array(data.M_atm)*data.grav/(4*np.pi*data.Rp**2)*1e-5
    ax[1].plot(data.time,ptot,c='k',lw=2)
    ax[1].set_xlabel("Time")
    ax[1].set_ylabel("Pressure in bar")

    plt.tight_layout()

    if data.saveplot:
      plt.savefig(data.output+'/plot_atm.png', dpi=100, transparent=False)
    if data.showplot:
      plt.show()
